In [3]:
import pandas as pd
from dbfread import DBF

# Reading the Counts CSV file into df1
df1 = pd.read_csv('final_vcount.csv')

# Reading the DBF file into df2
dbf_table = DBF('Veh_Network_StL_Calibration_v1.5.dbf')
df2 = pd.DataFrame(iter(dbf_table))

In [4]:
# This code cleans up the dbf 

# Filtering df1 to include only totals and the year 2021 counts
df1_filtered = df1[(df1['Classification'] == 'Totals') & (df1['Year'] == 2021)]

# Finding unique file names in df2 that are not in df1
df1_filt_unq = df1_filtered['File_name'].drop_duplicates()
df2_unq = df2['File_name'].drop_duplicates()
unmatched_file_names = df2_unq[~df2_unq.isin(df1_filt_unq)].tolist()

# Drop links with no intersection matches.
df2_filtered = df2[~df2['File_name'].isin(unmatched_file_names)]

print(unmatched_file_names)
print(df2.shape)
print(df2_filtered.shape)

df2_filtered.to_csv('StL_filtered.csv', index=False)

['N 33rd St & North Star_2019.xlsm', 'N 48th St & Target_2019.xlsm', 'S 14th St & YMCA_2020.xlsm', 'N Antelope Valley Pkwy & Military Rd_2019.xlsm', 'N Cotner Blvd & Vine St_2020.xlsm', 'S 70th St & Pine Lake Rd_2017.xlsx', 'N 48th St & Cornhusker Hwy _2022.xlsm', 'N 48th St & Cornhusker Hwy_2019.xlsm', 'NW 48th St & I80 WB_2017.xlsx', 'W 40th St & W O St_2018.xlsm', 'W 40th St & W O St_2018.xlsx', 'W 40th St & W O St_2022.xlsm', '56th St & O St_2019.xlsm', '56th St & O St_2022.xlsm', 'Capitol Pkwy & J St_2017.xlsx', 'S 33rd St & Hwy 2 _2017.xlsx', 'S 48th St & Normal Blvd_2019.xlsm', 'N 12th St & Q St_2017.xlsx', 'N 16th St & W St_2017.xlsx', 'N Antelope Valley Pkwy & Salt Creek Rdwy_2020.xlsm', 'N Antelope Valley Pkwy & Salt Creek Rdwy_2022.xlsm', 'Cotner Blvd & O St_2017.xlsx', 'N 48th St & Huntington Ave Ped Nov_2017.xlsm', 'N 48th St & Huntington Ave Ped Nov_2019.xlsm', 'N 48th St & Huntington Ave Ped Oct_2019.xlsm', 'S 13th St & Arapahoe St_2017.xlsx', 'S 13th St & Arapahoe St_20

In [5]:
# Joining receiving and approach ends for divided roadways in df2_filtered

variable_names = [
    'StL_0_0', 'StL_0_1', 'StL_0_2', 'StL_0_3', 'StL_0_4', 'StL_0_5', 
    'StL_1_0', 'StL_1_1', 'StL_1_2', 'StL_1_3', 'StL_1_4', 'StL_1_5', 
    'StL_2_0', 'StL_2_1', 'StL_2_2', 'StL_2_3', 'StL_2_4', 'StL_2_5', 
    'StL_3_0', 'StL_3_1', 'StL_3_2', 'StL_3_3', 'StL_3_4', 'StL_3_5', 
    'StL_4_0', 'StL_4_1', 'StL_4_2', 'StL_4_3', 'StL_4_4', 'StL_4_5', 
    'StL_5_0', 'StL_5_1', 'StL_5_2', 'StL_5_3', 'StL_5_4', 'StL_5_5', 
    'StL_6_0', 'StL_6_1', 'StL_6_2', 'StL_6_3', 'StL_6_4', 'StL_6_5', 
    'StL_7_0', 'StL_7_1', 'StL_7_2', 'StL_7_3', 'StL_7_4', 'StL_7_5'
]


# Custom conversion function to handle empty strings and extra precision
def conv(value):
    try:
        return float(value)
    except ValueError:
        return None  # indicating missing data

# Converting columns to numeric, handling empty strings and extra precision
df2_filtered[variable_names] = df2_filtered[variable_names].applymap(conv)

# Grouping by file name and direction by summing StL indicators
df2_grouped = df2_filtered.groupby(['File_name', 'Directio_1'])[variable_names].sum().reset_index()

# Now adding columns 34-40 and columns 89-268
merge_cols = ['File_name', 'Directio_1']
# Select columns 34 to 40 
cols_34_40 = df2_filtered.iloc[:, 33:40]
# Select columns 89 to 268
cols_89_268 = df2_filtered.iloc[:, 88:268]
retained_cols = pd.concat([cols_34_40, cols_89_268], axis=1)

df2_grouped = df2_grouped.merge(retained_cols, on = merge_cols, how='left')

# Drop duplicate rows, keeping only the first occurrence
#df2_grouped = df2_grouped.drop_duplicates(subset = merge_cols, keep='first')

print(df2_grouped.shape)

df2_grouped.to_csv('StL_filtered.csv', index=False)

C:\Users\msela\AppData\Local\Temp\ipykernel_17028\3280910193.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2_filtered[variable_names] = df2_filtered[variable_names].applymap(conv)
C:\Users\msela\AppData\Local\Temp\ipykernel_17028\3280910193.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_filtered[variable_names] = df2_filtered[variable_names].applymap(conv)


(2784, 235)


In [6]:
# Now introducing the Hour variable into df1 to make grouping 15-minute counts easier

# Creating a dictionary to map 'Start Time' to 'Hour'
hour_dict = {
    '12:00 AM': 1, '12:15 AM': 1, '12:30 AM': 1, '12:45 AM': 1,
    '1:00 AM': 2, '1:15 AM': 2, '1:30 AM': 2, '1:45 AM': 2,
    '2:00 AM': 3, '2:15 AM': 3, '2:30 AM': 3, '2:45 AM': 3,
    '3:00 AM': 4, '3:15 AM': 4, '3:30 AM': 4, '3:45 AM': 4,
    '4:00 AM': 5, '4:15 AM': 5, '4:30 AM': 5, '4:45 AM': 5,
    '5:00 AM': 6, '5:15 AM': 6, '5:30 AM': 6, '5:45 AM': 6,
    '6:00 AM': 7, '6:15 AM': 7, '6:30 AM': 7, '6:45 AM': 7,
    '7:00 AM': 8, '7:15 AM': 8, '7:30 AM': 8, '7:45 AM': 8,
    '8:00 AM': 9, '8:15 AM': 9, '8:30 AM': 9, '8:45 AM': 9,
    '9:00 AM': 10, '9:15 AM': 10, '9:30 AM': 10, '9:45 AM': 10,
    '10:00 AM': 11, '10:15 AM': 11, '10:30 AM': 11, '10:45 AM': 11,
    '11:00 AM': 12, '11:15 AM': 12, '11:30 AM': 12, '11:45 AM': 12,
    '12:00 PM': 13, '12:15 PM': 13, '12:30 PM': 13, '12:45 PM': 13,
    '1:00 PM': 14, '1:15 PM': 14, '1:30 PM': 14, '1:45 PM': 14,
    '2:00 PM': 15, '2:15 PM': 15, '2:30 PM': 15, '2:45 PM': 15,
    '3:00 PM': 16, '3:15 PM': 16, '3:30 PM': 16, '3:45 PM': 16,
    '4:00 PM': 17, '4:15 PM': 17, '4:30 PM': 17, '4:45 PM': 17,
    '5:00 PM': 18, '5:15 PM': 18, '5:30 PM': 18, '5:45 PM': 18,
    '6:00 PM': 19, '6:15 PM': 19, '6:30 PM': 19, '6:45 PM': 19,
    '7:00 PM': 20, '7:15 PM': 20, '7:30 PM': 20, '7:45 PM': 20,
    '8:00 PM': 21, '8:15 PM': 21, '8:30 PM': 21, '8:45 PM': 21,
    '9:00 PM': 22, '9:15 PM': 22, '9:30 PM': 22, '9:45 PM': 22,
    '10:00 PM': 23, '10:15 PM': 23, '10:30 PM': 23, '10:45 PM': 23,
    '11:00 PM': 24, '11:15 PM': 24, '11:30 PM': 24, '11:45 PM': 24
}

df1_filtered['Hour'] = df1_filtered['Start Time'].map(hour_dict)

C:\Users\msela\AppData\Local\Temp\ipykernel_17028\574343972.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_filtered['Hour'] = df1_filtered['Start Time'].map(hour_dict)


In [7]:
# Now assigning count volumes to directions for easier integration with df2_grouped

# Filter out rows where 'Approach' is not in the specified list
valid_approaches = ['Eastbound', 'Westbound', 'Northbound', 'Southbound']
df1_filtered = df1_filtered[df1_filtered['Approach'].isin(valid_approaches)]

# Convert columns to numeric
cols_convert = ['Right', 'Left', 'Thru', 'U-Turn']
df1_filtered[cols_convert] = df1_filtered[cols_convert].apply(pd.to_numeric)

def calc_direction (row):
    if row['Approach'] == 'Eastbound':
        W = row['Right'] + row['Left'] + row['Thru'] + 2*row['U-Turn']
        E = row['Thru']
        N = row['Left']
        S = row['Right']
    elif row['Approach'] == 'Westbound':
        E = row['Right'] + row['Left'] + row['Thru'] + 2*row['U-Turn']
        W = row['Thru']
        S = row['Left']
        N = row['Right']
    elif row['Approach'] == 'Northbound':
        S = row['Right'] + row['Left'] + row['Thru'] + 2*row['U-Turn']
        N = row['Thru']
        E = row['Right']
        W = row['Left']
    elif row['Approach'] == 'Southbound':
        N = row['Right'] + row['Left'] + row['Thru'] + 2*row['U-Turn']
        S = row['Thru']
        W = row['Right']
        E = row['Left']
    return pd.Series([E, W, N, S])

# Apply the function to the DataFrame
df1_filtered[['E', 'W', 'N', 'S']] = df1_filtered.apply(calc_direction, axis=1)

df1_filtered.to_csv('counts_filtered.csv', index=False)

In [8]:
# Grouping counts by intersections (File_name) and hours

df1_grouped = df1_filtered.groupby(['File_name', 'Hour'])[['E', 'W', 'N', 'S']].sum().reset_index()

df1_grouped.to_csv('counts_final.csv', index=False)

In [9]:
import numpy as np

# Create dataframe 2 with 24 Count columns initialized to NaN
for hour in range(1, 25):
    df2_grouped[f'Count_{hour}'] = np.nan

# Iterate over rows in df2_grouped
for index, row in df2_grouped.iterrows():
    file_name = row['File_name']
    direction = row['Directio_1']
    
    # Find corresponding row in df1_grouped
    find_row = df1_grouped[df1_grouped['File_name'] == file_name]
    
    # Iterate over the 24 hours
    for hour in range(1, 25):
        # Extract value from df1 based on conditions
        value = find_row.loc[find_row['Hour'] == hour, direction].values
        if len(value) > 0:
            df2_grouped.at[index, f'Count_{hour}'] = value[0]
            
# Now assigning Year, Month and Day_of_Week 
df1_DOW = df1_filtered[['File_name', 'Day_of_Week']].drop_duplicates(subset='File_name')
# Merge df2_grouped with the unique entries from df1_grouped
df2_grouped = df2_grouped.merge(df1_DOW, on='File_name', how='left')

# Display resulting dataframe 2
print(df2_grouped)


                                           File_name Directio_1  StL_0_0  \
0                           14th St & O St_2021.xlsm          E  16092.2   
1                           14th St & O St_2021.xlsm          N   1344.0   
2                           14th St & O St_2021.xlsm          S   2872.6   
3                           14th St & O St_2021.xlsm          W  16092.2   
4                            1st St & O St_2021.xlsm          E  41796.4   
...                                              ...        ...      ...   
2779  Wilderness Ridge Dr & Yankee Hill Rd_2021.xlsm          N   2772.4   
2780  Wilderness Ridge Dr & Yankee Hill Rd_2021.xlsm          N   2772.4   
2781  Wilderness Ridge Dr & Yankee Hill Rd_2021.xlsm          N   2772.4   
2782  Wilderness Ridge Dr & Yankee Hill Rd_2021.xlsm          W  18538.0   
2783  Wilderness Ridge Dr & Yankee Hill Rd_2021.xlsm          W  18538.0   

      StL_0_1  StL_0_2  StL_0_3  StL_0_4  StL_0_5  StL_1_0  StL_1_1  ...  \
0      2653

In [13]:
# Now defining day parts (0 to 5) for counts according to StL day parts

# Initializing Count_dp_{i} variables
for i in [0, 1, 3, 4, 5]:
    df2_grouped[f'Count_dp_{i}'] = np.nan
    
# Now assigning volumes to the Counts day parts based on their availabilities

# This is for day part 0, which requires all hours to have counts
df2_grouped['Count_dp_0'] = df2_grouped[[f'Count_{i}' for i in range(1, 25)]].apply(lambda row: row.sum() if row.notna().all() else np.nan, axis=1
)

# day part 1 (12AM to 6AM)
df2_grouped['Count_dp_1'] = df2_grouped[[f'Count_{i}' for i in range(1, 7)]].apply(lambda row: row.sum() if row.notna().all() else np.nan, axis=1
)

# day part 2 (6AM to 10AM)
df2_grouped['Count_dp_2'] = df2_grouped[[f'Count_{i}' for i in range(7, 11)]].apply(lambda row: row.sum() if row.notna().all() else np.nan, axis=1
)

# day part 3 (10AM to 3PM)
df2_grouped['Count_dp_3'] = df2_grouped[[f'Count_{i}' for i in range(11, 16)]].apply(lambda row: row.sum() if row.notna().all() else np.nan, axis=1
)

# day part 4 (3PM to 7PM)
df2_grouped['Count_dp_4'] = df2_grouped[[f'Count_{i}' for i in range(16, 20)]].apply(lambda row: row.sum() if row.notna().all() else np.nan, axis=1
)

# day part 5 (7PM to 12AM)
df2_grouped['Count_dp_5'] = df2_grouped[[f'Count_{i}' for i in range(20, 25)]].apply(lambda row: row.sum() if row.notna().all() else np.nan, axis=1
)

In [14]:
# Finalizing the StL variable and the counts dependent variable for each row.

# Count variable
def calculate_count_vol(x):
    if pd.notna(x['Count_dp_0']):
        return x['Count_dp_0']
    else:
        return x[['Count_dp_1', 'Count_dp_2', 'Count_dp_3', 'Count_dp_4', 'Count_dp_5']].sum(skipna=True)

df2_grouped['Count_vol'] = df2_grouped.apply(calculate_count_vol, axis=1)

In [15]:
# Now calculating the StL volume that corresponds to the available counts

# Dictionary to map day of week to the number in the StL variables
day_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

# Define a function to calculate StL_x, which is the explanatory StL variable in the model
def calculate_StL_x(row):
    day_num = day_mapping[row['Day_of_Week']]
    if pd.notna(row['Count_dp_0']):
        return row[f'StL_{day_num}_0']  # returns all day StL for the corresponding day if 24 counts are available
    else:
        counts = row[['Count_dp_1', 'Count_dp_2', 'Count_dp_3', 'Count_dp_4', 'Count_dp_5']]
        # Select the corresponding StL columns
        stl_cols = [f'StL_{day_num}_{i}' for i in range(1, 6)]
        stl_values = row[stl_cols]
        # Sum the non-NaN values for the StL values corresponding the same day for day parts 1 to 5
        valid_stl_values = stl_values[counts.notna().values]
        return valid_stl_values.sum()

# Apply the function to each row to create the 'StL_x' column
df2_grouped['StL_x'] = df2_grouped.apply(calculate_StL_x, axis=1)

df2_grouped.to_csv('vcalibration_input.csv', index=False)